In [18]:
import snowflake.connector


In [19]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [20]:
from dotenv import load_dotenv, find_dotenv
import os

# Find and load the nearest .env (project root)
load_dotenv(find_dotenv(), override=True)

def need(name):
    v = os.getenv(name)
    if not v:
        raise RuntimeError(f"Missing required env var: {name}")
    # remove accidental quotes/spaces
    return v.strip().strip('"').strip("'")

SNOWFLAKE_USER     = need("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = need("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT  = need("SNOWFLAKE_ACCOUNT")  # e.g., afaypkh-rjb48354


In [34]:
import snowflake.connector

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
)

In [22]:
cs = conn.cursor()

In [23]:
cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")

In [24]:
cs.execute("CREATE DATABASE IF NOT EXISTS testdb")

In [26]:
from pathlib import Path
import re

# ---- 0) connect  ----
conn = snowflake.connector.connect(
    host="qbmhuza-bnb86629.snowflakecomputing.com",   
    account="qbmhuza-bnb86629",                        
    user="second2",
    password="gyczeg6kaHqywownor",
    warehouse="COMPUTE_WH",                           
)
cs = conn.cursor()

# make sure the warehouse will wake itself
cs.execute("ALTER WAREHOUSE COMPUTE_WH SET AUTO_SUSPEND=60 AUTO_RESUME=TRUE")

# ---- 1) set DB/Schema (use your existing TESTDB) ----
DB, SCHEMA = "TESTDB", "PUBLIC"
cs.execute(f"CREATE DATABASE IF NOT EXISTS {DB}")
cs.execute(f"CREATE SCHEMA   IF NOT EXISTS {DB}.{SCHEMA}")
cs.execute(f"USE DATABASE {DB}")
cs.execute(f"USE SCHEMA {SCHEMA}")

# ---- 2) read and run your .pgsql file from the repo ----
sql_path = Path("Data/supplier_case.pgsql")     
assert sql_path.exists(), f"Not found: {sql_path.resolve()}"
txt = sql_path.read_text(encoding="utf-8")

# tiny Postgres -> Snowflake cleanups
txt = "\n".join(l for l in txt.splitlines() if not l.strip().startswith("\\"))            # drop psql meta commands
txt = re.sub(r"\bNUMERIC\b", "NUMBER", txt, flags=re.I)                                   # NUMERIC -> NUMBER (safe)
txt = re.sub(r"\bsupplier_case\b", f"{DB}.{SCHEMA}.SUPPLIER_CASE", txt, flags=re.I)       # fully-qualify table

# split on semicolons and execute
stmts = [s.strip() for s in re.split(r";\s*(?=\n|$)", txt) if s.strip()]
for s in stmts:
    cs.execute(s)

# ---- 3) visualize (still only cs.execute) ----
print("Rows:", cs.execute(f"SELECT COUNT(*) FROM {DB}.{SCHEMA}.SUPPLIER_CASE").fetchone()[0])

print("\nSample rows:")
for r in cs.execute(f"""
    SELECT SupplierID, SupplierName, PhoneNumber, WebsiteURL,
           TRY_TO_DATE(ValidFrom) AS ValidFrom, TRY_TO_DATE(ValidTo) AS ValidTo
    FROM {DB}.{SCHEMA}.SUPPLIER_CASE
    ORDER BY SupplierID
    LIMIT 10
""").fetchall():
    print(r)

print("\nSchema:")
for r in cs.execute(f"DESCRIBE TABLE {DB}.{SCHEMA}.SUPPLIER_CASE").fetchall():
    print(r[0], r[1])


Rows: 13

Sample rows:
(1, 'A Datum Corporation', '(847) 555-0100', 'http://www.adatum.com', None, None)
(2, 'Contoso, Ltd.', '(360) 555-0100', 'http://www.contoso.com', None, None)
(3, 'Consolidated Messenger', '(415) 555-0100', 'http://www.consolidatedmessenger.com', None, None)
(4, 'Fabrikam, Inc.', '(203) 555-0104', 'http://www.fabrikam.com', None, None)
(5, 'Graphic Design Institute', '(406) 555-0105', 'http://www.graphicdesigninstitute.com', None, None)
(6, 'Humongous Insurance', '(423) 555-0105', 'http://www.humongousinsurance.com', None, None)
(7, 'Litware, Inc.', '(209) 555-0108', 'http://www.litwareinc.com', None, None)
(8, 'Lucerne Publishing', '(423) 555-0103', 'http://www.lucernepublishing.com', None, None)
(9, 'Nod Publishers', '(252) 555-0100', 'http://www.nodpublishers.com', None, None)
(10, 'Northwind Electric Cars', '(201) 555-0105', 'http://www.northwindelectriccars.com', None, None)

Schema:
SUPPLIERID NUMBER(38,0)
SUPPLIERNAME VARCHAR(16777216)
SUPPLIERCATEGORYID N

In [27]:
cs.execute("USE DATABASE TESTDB")
cs.execute("USE SCHEMA PUBLIC")

cs.execute("""
CREATE OR REPLACE TABLE TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN AS
SELECT
  CAST(SUPPLIERID               AS INT)        AS SUPPLIERID,
  SUPPLIERNAME                                   AS SUPPLIERNAME,
  CAST(SUPPLIERCATEGORYID       AS INT)        AS SUPPLIERCATEGORYID,
  CAST(PRIMARYCONTACTPERSONID   AS INT)        AS PRIMARYCONTACTPERSONID,
  CAST(ALTERNATECONTACTPERSONID AS INT)        AS ALTERNATECONTACTPERSONID,
  CAST(DELIVERYMETHODID         AS INT)        AS DELIVERYMETHODID,
  CAST(POSTALCITYID             AS INT)        AS POSTALCITYID,
  SUPPLIERREFERENCE                              AS SUPPLIERREFERENCE,
  PHONENUMBER                                   AS PHONENUMBER,
  WEBSITEURL                                    AS WEBSITEURL,
  DELIVERYADDRESSLINE1                           AS DELIVERYADDRESSLINE1,
  CAST(DELIVERYPOSTALCODE       AS INT)        AS DELIVERYPOSTALCODE,
  POSTALADDRESSLINE1                             AS POSTALADDRESSLINE1,
  CAST(POSTALPOSTALCODE         AS INT)        AS POSTALPOSTALCODE,
  CAST(LASTEDITEDBY             AS INT)        AS LASTEDITEDBY,
  TRY_TO_DATE(VALIDFROM)                        AS VALIDFROM,
  TRY_TO_DATE(VALIDTO)                          AS VALIDTO
FROM TESTDB.PUBLIC.SUPPLIER_CASE;
""")


In [28]:
print(cs.execute("SELECT COUNT(*) FROM TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN").fetchone()[0])
for r in cs.execute("""
  SELECT SUPPLIERID, SUPPLIERNAME, PHONENUMBER, WEBSITEURL, VALIDFROM, VALIDTO
  FROM TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN
  ORDER BY SUPPLIERID
  LIMIT 10
""").fetchall():
    print(r)


13
(1, 'A Datum Corporation', '(847) 555-0100', 'http://www.adatum.com', None, None)
(2, 'Contoso, Ltd.', '(360) 555-0100', 'http://www.contoso.com', None, None)
(3, 'Consolidated Messenger', '(415) 555-0100', 'http://www.consolidatedmessenger.com', None, None)
(4, 'Fabrikam, Inc.', '(203) 555-0104', 'http://www.fabrikam.com', None, None)
(5, 'Graphic Design Institute', '(406) 555-0105', 'http://www.graphicdesigninstitute.com', None, None)
(6, 'Humongous Insurance', '(423) 555-0105', 'http://www.humongousinsurance.com', None, None)
(7, 'Litware, Inc.', '(209) 555-0108', 'http://www.litwareinc.com', None, None)
(8, 'Lucerne Publishing', '(423) 555-0103', 'http://www.lucernepublishing.com', None, None)
(9, 'Nod Publishers', '(252) 555-0100', 'http://www.nodpublishers.com', None, None)
(10, 'Northwind Electric Cars', '(201) 555-0105', 'http://www.northwindelectriccars.com', None, None)


In [32]:
# --- Add this to ETL_Snowflake.py (after your existing connection code) ---

import snowflake.connector

def use_first_existing_db(cs, candidate_names):
    """
    Try each candidate database name; return the first one that exists.
    Falls back to SHOW DATABASES LIKE 'WEATHER%ENVIRONMENT%' if none match exactly.
    """
    # Try exact names first
    for db in candidate_names:
        cs.execute(f"SHOW DATABASES LIKE '{db}'")
        if cs.fetchone():
            return db

    # Broad match as a fallback (handles custom naming)
    cs.execute("SHOW DATABASES LIKE 'WEATHER%ENVIRONMENT%'")
    row = cs.fetchone()
    if row:
        # SHOW DATABASES returns tuples; the 2nd field is the database NAME
        # Typical order: created_on, name, is_default, ...
        return row[1]

    raise RuntimeError(
        "Could not find a WEATHER...ENVIRONMENT database. "
        "Verify the Snowflake Marketplace subscription/name in the web UI."
    )

def print_table_sample(cs, fully_qualified_table, sample_rows=5):
    """
    Print column names, a small sample, and total row count for the given table.
    """
    print(f"\n=== {fully_qualified_table} ===")

    # Sample rows
    cs.execute(f"SELECT * FROM {fully_qualified_table} LIMIT {sample_rows}")
    rows = cs.fetchall()
    col_names = [desc[0] for desc in cs.description]
    print("Columns:", ", ".join(col_names))
    for i, r in enumerate(rows, 1):
        print(f"{i:>2}: {r}")

    # Row count (fast metadata-style count is not available; run COUNT(*))
    cs.execute(f"SELECT COUNT(*) FROM {fully_qualified_table}")
    total = cs.fetchone()[0]
    print(f"Total rows: {total}")

def print_cybersyn_weather_tables(conn, warehouse="COMPUTE_WH"):
    """
    Uses only the Snowflake Python connector to:
    - set the warehouse,
    - locate the Marketplace database,
    - switch to CYBERSYN schema,
    - and print samples + counts from the two NOAA tables.
    """
    with conn.cursor() as cs:
        # Ensure a warehouse is set (you created MY_FIRST_WAREHOUSE earlier in your file)
        cs.execute(f"USE WAREHOUSE {warehouse}")

        # Part 2, step 7 database names: assignment shows WEATHER__ENVIRONMENT (double underscore),
        # but some accounts show WEATHER_ENVIRONMENT (single underscore).
        candidate_dbs = ["WEATHER__ENVIRONMENT", "WEATHER_ENVIRONMENT"]
        db_name = use_first_existing_db(cs, candidate_dbs)
        cs.execute(f"USE DATABASE {db_name}")

        # Schema from the brief is CYBERSYN
        cs.execute("USE SCHEMA CYBERSYN")

        # Fully qualified table names (no quotes needed since identifiers are simple/upper)
        tables = [
            "NOAA_WEATHER_METRICS_TIMESERIES",
            "NOAA_WEATHER_STATION_INDEX",
        ]
        for t in tables:
            fqtn = f"{db_name}.CYBERSYN.{t}"
            print_table_sample(cs, fqtn, sample_rows=5)


# --- Run it ---
if __name__ == "__main__":
    # Reuse your existing `conn` from above in ETL_Snowflake.py
    # Example assumes you already did:
    # conn = snowflake.connector.connect(user=..., password=..., account=...)
    try:
        print_cybersyn_weather_tables(conn, warehouse="COMPUTE_WH")
    finally:
        try:
            conn.close()
        except Exception:
            pass



=== WEATHER__ENVIRONMENT.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES ===
Columns: NOAA_WEATHER_STATION_ID, VARIABLE, VARIABLE_NAME, DATE, DATETIME, VALUE, UNIT
 1: ('CA007011309', 'precipitation', 'Precipitation', datetime.date(2007, 4, 22), datetime.datetime(2007, 4, 22, 0, 0), Decimal('0.000000'), 'Millimeters')
 2: ('USC00046826', 'maximum_temperature', 'Maximum Temperature', datetime.date(2002, 10, 21), datetime.datetime(2002, 10, 21, 16, 0), Decimal('23.900000'), 'Degrees Celsius')
 3: ('USC00426708', 'temperature_at_observation_time', 'Temperature at Observation Time', datetime.date(2002, 11, 18), datetime.datetime(2002, 11, 18, 19, 0), Decimal('3.300000'), 'Degrees Celsius')
 4: ('USR0000ISPA', 'average_temperature', 'Average Temperature', datetime.date(2016, 4, 3), datetime.datetime(2016, 4, 3, 0, 0), Decimal('11.900000'), 'Degrees Celsius')
 5: ('CA001018620', 'precipitation', 'Precipitation', datetime.date(2010, 12, 9), datetime.datetime(2010, 12, 9, 0, 0), Decimal('15.800000')

In [38]:
import snowflake.connector

conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
)
cs = conn.cursor()

In [67]:
import os, glob
from pathlib import Path

## Creating the PO_Table with Datatypes 
cs.execute(f"USE DATABASE {DB}")
cs.execute(f"USE SCHEMA {SCHEMA}")
cs.execute(
"CREATE OR REPLACE TABLE PO_Data("
"purchaseorderid NUMBER(38,0), "
"supplierid NUMBER(38,0), "
"orderdate DATE, "
"deliverymethodid NUMBER(38,0), "
"contactpersonid NUMBER(38,0), "
"expecteddeliverydate DATE, "
"supplierreference VARCHAR, "
"isorderfinalized NUMBER(1,0), "
"comments VARCHAR, "
"internalcomments VARCHAR, "
"lasteditedby NUMBER(38,0), "
"purchaseorderlineid NUMBER(38,0), "
"stockitemid NUMBER(38,0), "
"orderedouters NUMBER(38,0), "
"description VARCHAR, "
"receivedouters NUMBER(38,0), "
"packagetypeid NUMBER(38,0), "
"expectedunitpriceperouter NUMBER(18,4), "
"lastreceiptdate DATE, "
"isorderlinefinalized NUMBER(1,0), "
"right_lasteditedby NUMBER(38,0), "
"right_lasteditedwhen TIMESTAMP_NTZ"
")")

# ---------- Resolve repo-relative data folder ----------
def find_monthly_po_dir() -> Path:
    """
    Locate the 'Data/Monthly PO Data' folder relative to the repository.
    Works from notebooks or scripts, on Windows/macOS/Linux.
    """
    candidates = [
        Path.cwd() / "Data" / "Monthly PO Data",
        Path.cwd() / "data" / "Monthly PO Data",
    ]

    # If running from a subfolder, search upward then rglob for the directory
    # 1) Walk up to (at most) 5 levels to find a '.git' folder (repo root)
    here = Path.cwd()
    ups = [here] + list(here.parents)[:5]
    repo_roots = [p for p in ups if (p / ".git").exists()]
    roots_to_search = repo_roots[:1] or [here]

    for root in roots_to_search:
        candidates.append(root / "Data" / "Monthly PO Data")
        candidates.append(root / "data" / "Monthly PO Data")
        # fallback: recursive search for the exact folder name
        for p in root.rglob("Monthly PO Data"):
            candidates.append(p)

    for p in candidates:
        if p.exists() and p.is_dir():
            # Must contain CSVs to be considered valid
            if any(p.glob("*.csv")):
                return p

    raise SystemExit("Could not find 'Data/Monthly PO Data' in this repo. "
                     "Make sure the data folder exists and contains .csv files.")

local_dir_path = find_monthly_po_dir()
local_dir = str(local_dir_path)  # keep your existing code style
print("Using data folder:", local_dir)

# ---------- Stage + file format ----------


cs.execute("CREATE OR REPLACE STAGE po_data_stage")
cs.execute("""
CREATE OR REPLACE FILE FORMAT po_csv_ff
  TYPE=CSV
  FIELD_DELIMITER=','
  FIELD_OPTIONALLY_ENCLOSED_BY='"'
  SKIP_HEADER=1
  TRIM_SPACE=TRUE
  EMPTY_FIELD_AS_NULL=TRUE
  NULL_IF=('','NULL','null','00:00.0','0:00.0','00:00','0:00')
  DATE_FORMAT='AUTO'
  TIME_FORMAT='AUTO'
  TIMESTAMP_FORMAT='AUTO'
""")

# ---------- Local files to stage (repo-relative) ----------

pattern = os.path.join(local_dir, "*.csv")
files = glob.glob(pattern)
print("Matched CSVs:", len(files))
if not files:
    raise SystemExit(f"No CSVs matched at: {pattern}")

# ---------- PUT files into stage (auto-compress -> .gz) ----------
for filepath in files:
    base = os.path.basename(filepath)
    if ":" in base:   # skip Windows ADS like ':Zone.Identifier'
        continue
    abs_path = os.path.abspath(filepath).replace("\\", "/")   # ensure forward slashes
    file_uri = "file:///" + abs_path.lstrip("/")              # exactly 3 slashes, no URL-encoding
    print("PUT ->", file_uri)
    cs.execute(f"PUT '{file_uri}' @po_data_stage AUTO_COMPRESS=TRUE OVERWRITE=TRUE")
    

# --- sanity check what's in the stage ---
cs.execute("LIST @po_data_stage")
print("Staged objects (top 10):", cs.fetchall()[:10])

# --- load into the table (skipping $12 = lasteditedwhen) ---
cs.execute("""
COPY INTO PO_Data
  FROM (
    SELECT
      $1  ::NUMBER(38,0)  AS purchaseorderid,
      $2  ::NUMBER(38,0)  AS supplierid,
      TRY_TO_DATE($3)     AS orderdate,
      $4  ::NUMBER(38,0)  AS deliverymethodid,
      $5  ::NUMBER(38,0)  AS contactpersonid,
      TRY_TO_DATE($6)     AS expecteddeliverydate,
      $7                  AS supplierreference,
      $8  ::NUMBER(1,0)   AS isorderfinalized,
      $9                  AS comments,
      $10                 AS internalcomments,
      $11 ::NUMBER(38,0)  AS lasteditedby,
      /* skip $12 */
      $13 ::NUMBER(38,0)  AS purchaseorderlineid,
      $14 ::NUMBER(38,0)  AS stockitemid,
      $15 ::NUMBER(38,0)  AS orderedouters,
      $16                 AS description,
      $17 ::NUMBER(38,0)  AS receivedouters,
      $18 ::NUMBER(38,0)  AS packagetypeid,
      $19 ::NUMBER(18,4)  AS expectedunitpriceperouter,
      TRY_TO_DATE($20)    AS lastreceiptdate,
      $21 ::NUMBER(1,0)   AS isorderlinefinalized,
      $22 ::NUMBER(38,0)  AS right_lasteditedby,
      TRY_TO_TIMESTAMP_NTZ($23) AS right_lasteditedwhen
    FROM @po_data_stage (FILE_FORMAT => 'po_csv_ff')
  )
  ON_ERROR = ABORT_STATEMENT
""")

Using data folder: /home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data
Matched CSVs: 41
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2019-6.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2021-10.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2020-1.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2022-3.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2021-1.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2020-8.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Data/Monthly PO Data/2021-7.csv
PUT -> file:///home/jovyan/UCSD CLASSES/MGTA 464- SQL/MGTA_464_Snowflake_Project/Dat

In [68]:
cs.execute("SELECT COUNT(*) FROM PO_Data")
print("Row count:", cs.fetchone()[0])
print("Row count should equal 8367")


Row count: 8367
Row count should equal 8367


In [58]:
cs.execute("SELECT * FROM PO_Data LIMIT 10")
for row in cs.fetchall():
    print(row)


(106, 4, datetime.date(2019, 3, 1), 7, 2, datetime.date(2019, 3, 21), '293092', 1, None, None, 4, 469, 77, 92, '"The Gu" red shirt XML tag t-shirt (White) XXS', 92, 6, Decimal('84.0000'), datetime.date(2019, 3, 4), 1, 4, None)
(106, 4, datetime.date(2019, 3, 1), 7, 2, datetime.date(2019, 3, 21), '293092', 1, None, None, 4, 470, 78, 127, '"The Gu" red shirt XML tag t-shirt (White) XS', 127, 6, Decimal('84.0000'), datetime.date(2019, 3, 4), 1, 4, None)
(106, 4, datetime.date(2019, 3, 1), 7, 2, datetime.date(2019, 3, 21), '293092', 1, None, None, 4, 471, 80, 20, '"The Gu" red shirt XML tag t-shirt (White) M', 20, 6, Decimal('84.0000'), datetime.date(2019, 3, 4), 1, 4, None)
(106, 4, datetime.date(2019, 3, 1), 7, 2, datetime.date(2019, 3, 21), '293092', 1, None, None, 4, 472, 86, 74, '"The Gu" red shirt XML tag t-shirt (White) 5XL', 74, 6, Decimal('96.0000'), datetime.date(2019, 3, 4), 1, 4, None)
(106, 4, datetime.date(2019, 3, 1), 7, 2, datetime.date(2019, 3, 21), '293092', 1, None, None

1) PO totals (POAmount) and a tidy PO header table

In [69]:
# One row per purchase order with the required total
cs.execute("""
CREATE OR REPLACE VIEW PO_Header AS
SELECT
  purchaseorderid,
  MIN(orderdate)                 AS orderdate,
  MIN(supplierid)                AS supplierid,
  SUM(receivedouters * expectedunitpriceperouter) AS POAmount
FROM PO_Data
GROUP BY purchaseorderid;
""")
